In [42]:
from bs4 import BeautifulSoup
import requests
import time, os
import re

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [3]:
url = 'https://shop.rebag.com/collections/all-bags?_=pf&pf_st_availability_hidden=true&pf_t_categories=bc-filter-Bags'

In [4]:
driver = webdriver.Chrome(chromedriver)


In [5]:
url_single = 'https://shop.rebag.com/collections/all-bags/products/handbags-christian-dior-saddle-handbag-leather-mini59393229'
driver.get(url_single)

soup_single = BeautifulSoup(driver.page_source, 'html.parser')

In [6]:
print(soup_single.prettify())

<html class="no-js" lang="en">
 <!--<![endif]-->
 <link href="//cdn.shopify.com/s/files/1/0384/0161/t/423/assets/app.css?v=12641472676068445895" rel="stylesheet"/>
 <link href="//cdn.shopify.com/s/files/1/0384/0161/t/423/assets/app-compatibility.css?v=16380685170809679860" rel="stylesheet"/>
 <head>
  <meta charset="utf-8"/>
  <!--[if IE]><meta http-equiv='X-UA-Compatible' content='IE=edge,chrome=1'/><![endif]-->
  <title>
   Christian Dior Saddle Handbag Leather Mini 59393229 - Rebag
  </title>
  <link href="//cdn.shopify.com/s/files/1/0384/0161/t/423/assets/favicon.png?v=18389839475860471540" rel="icon" type="image/png"/>
  <noscript>
   <link href="//cdn.shopify.com/s/files/1/0384/0161/t/423/assets/app.min.css?v=10398556964625610473" rel="stylesheet" type="text/css"/>
  </noscript>
  <meta content="Online Sale - Authentic Red Christian Dior Saddle Handbag Leather Mini at Rebag.com. Guaranteed genuine! Financing available. 59393229" name="description"/>
  <link href="https://shop.reb

In [7]:
title_string_single = soup_single.find('title').text
title_single = title_string_single.split('-')[0].strip()

title_single = title_single.rsplit(' ', 1)[0]
title_single

'Christian Dior Saddle Handbag Leather Mini'

In [8]:
price_single = soup_single.find(class_='pdp__price').text
price_single

'$2,345USD'

In [9]:
ec_string = soup_single.find(text='Exterior Color')  

In [10]:
ec_string.findNext().text

'Red'

In [11]:
def get_details_value(soup, field_name):
    
    '''Grab a value from Box Office Mojo HTML
    
    Takes a string attribute of a movie on the page and returns the string in
    the next sibling object (the value for that attribute) or None if nothing is found.
    '''
    
    obj = soup.find(text=field_name)
    
    if not obj: 
        return None
    
    # this works for most of the values
    next_element = obj.findNext()
    
    if next_element:
        return next_element.text 
    else:
        return None

In [37]:
ex_color = get_details_value(soup_single,'Exterior Color')
print(ex_color)

Red


In [12]:
ex_material = get_details_value(soup_single,'Exterior Material')
print(ex_material)

Leather


In [13]:
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [14]:
print(soup.prettify())

<html class="no-js" lang="en">
 <!--<![endif]-->
 <link href="//cdn.shopify.com/s/files/1/0384/0161/t/423/assets/app.css?v=12641472676068445895" rel="stylesheet"/>
 <link href="//cdn.shopify.com/s/files/1/0384/0161/t/423/assets/app-compatibility.css?v=16380685170809679860" rel="stylesheet"/>
 <head>
  <meta charset="utf-8"/>
  <!--[if IE]><meta http-equiv='X-UA-Compatible' content='IE=edge,chrome=1'/><![endif]-->
  <title>
   Shop Authentic, Pre-Owned Handbags - Rebag
  </title>
  <link href="//cdn.shopify.com/s/files/1/0384/0161/t/423/assets/favicon.png?v=18389839475860471540" rel="icon" type="image/png"/>
  <noscript>
   <link href="//cdn.shopify.com/s/files/1/0384/0161/t/423/assets/app.min.css?v=10398556964625610473" rel="stylesheet" type="text/css"/>
  </noscript>
  <meta content="Shop all handbags online at rebag.com. We offer pre-loved, authentic luxury handbags for sale now. Financing and returns available." name="description"/>
  <link href="https://shop.rebag.com/collections/a

In [30]:
rows = soup.find_all(class_='product-caption') #.find('a')

In [31]:
len(rows)

48

In [32]:
#s['href']

In [33]:
link_clothing = []
for row in rows:
    link_clothing.append(row.find('a')['href'])

In [41]:
link_clothing[3]

'/collections/all-bags/products/handbags-christian-dior-saddle-handbag-leather-medium61020346'

List of all clothing, let's try open all pages from this page

In [38]:
def get_cloth_dict(link):
    '''
    From BoxOfficeMojo link stub, request movie html, parse with BeautifulSoup, and
    collect 
        - title 
        - domestic gross
        - runtime 
        - MPAA rating
        - full release date
    Return information as a dictionary.
    '''
    
    base_url = 'https://shop.rebag.com'
    
    #Create full url to scrape
    url = base_url + link
    
    #Request HTML and parse
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    #title
    title_string = soup.find('title').text
    title = title_string.split('-')[0].strip()
    title = title.rsplit(' ', 1)[0]
    print(title)

    price = soup.find(class_='pdp__price').text   
    print(price)
    
    ex_color = get_details_value(soup,'Exterior Color')
    print(ex_color)
    
    ex_material = get_details_value(soup,'Exterior Material')
    print(ex_material)

    return title, price

In [39]:
for l in link_clothing[0:10]:
    print(get_cloth_dict(l))

Christian Dior Saddle Handbag Leather Mini
$2,345USD
Red
Leather
('Christian Dior Saddle Handbag Leather Mini', '$2,345USD')
Chanel Boy Flap Bag Quilted Caviar Old Medium
$5,350USD
Pink
Leather
('Chanel Boy Flap Bag Quilted Caviar Old Medium', '$5,350USD')
Chanel Classic Single Flap Bag Quilted Denim with Tweed Mini


AttributeError: 'NoneType' object has no attribute 'text'